In [26]:
# Numpy and Pandas
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from numpy import mean
from numpy import std
import time
# Iterative Stratification untuk cross validation multilabel
from skmultilearn.model_selection import IterativeStratification
#Import Tensorflow dan extension
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization, Activation
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.callbacks import  EarlyStopping
from tensorflow.keras import backend as K
from tensorflow.keras.backend import sigmoid
from tensorflow.keras.optimizers import Adam, Nadam, Adagrad, SGD, RMSprop, Adadelta
import xgboost as xgb
#Import keras tuner dan metrics untuk tuning parameter
import kerastuner as kt
from kerastuner.tuners import RandomSearch, BayesianOptimization, Sklearn
from sklearn import metrics
import tensorflow_addons as tfa

#
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import PrecisionRecallDisplay
from sklearn.metrics import average_precision_score

In [2]:
#Modifikasi IterativeStratification agar hasil random data tetap sama

def new_init(self, n_splits=3, order=1, sample_distribution_per_fold = None, random_state=None):

                  self.order = order
                  if random_state is not None:
                      do_shuffle = True
                  else:
                      do_shuffle = False
                  super(
                      IterativeStratification,
                      self).__init__(n_splits,
                                     shuffle=do_shuffle,
                                     random_state=random_state)
                  if sample_distribution_per_fold:
                      self.percentage_per_fold = sample_distribution_per_fold
                  else:
                      self.percentage_per_fold = [1 / float(self.n_splits) for _ in range(self.n_splits)]
    
IterativeStratification.__init__ = new_init

In [20]:
#Import Dataset
dataset_pepi = pd.read_csv('E:/Projek Pak Wisnu/Projek Peptide-Protein Interaction/dataset pepPi.csv')
# dataset_circular = pd.read_csv('E:/Projek pak Sony/com_fp_circular.csv')
# dataset_maccs = pd.read_csv('E:/Projek pak Sony/com_fp_maccs.csv')
kelasdata = pd.read_csv('E:/Projek Pak Wisnu/Projek Peptide-Protein Interaction/kelasdatacomplete.csv')
X_pepi = dataset_pepi.drop("pdb_chain", axis =1 )
# X_cl = dataset_circular.drop("CID",axis=1)
# X_mc = dataset_maccs.drop("CID",axis=1)
Y = np.array(kelasdata)
X_pepi
kelasdata

,Unknown0_L,Unknown1_A,Unknown2_A,Unknown3_A,Unknown4_B,Unknown5_H,Unknown6_L,Unknown7_H,Unknown8_L,Unknown9_A,...,A7MB62_B,Q58CQ2_C,Q3MHR7_D,Q3SYX9_G,Q148J6_F,Q3T035_E,P83111_A,Q946J8_A,Q8CJ12_R,Q03164_B
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6985,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
6986,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6987,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [25]:
X_pepi

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,-3.555556,-0.222222,-9.388889,-9.388889,-12.500000,-0.888889,-2.722222,-0.055556,-1.388889,-0.888889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.726331,-0.998028,-2.879684,-1.140533,-2.390533,-1.181460,-1.501479,-0.825444,-2.584813,-1.642505,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.252232,-0.392857,-3.330357,-4.187500,-3.859375,-0.859375,-1.758929,-2.794643,-1.698661,-0.937500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.197279,-0.394558,-3.935374,-5.292517,-5.217687,-0.833333,-1.965986,-1.673469,-0.513605,-1.336735,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.252232,-0.392857,-3.330357,-4.187500,-3.859375,-0.859375,-1.758929,-2.794643,-1.698661,-0.937500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,-0.417432,0.147673,-1.082674,-0.412156,-2.529018,-0.221325,-0.857761,-0.446713,-0.805054,-0.704706,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6985,-0.778409,-1.131313,-2.335859,-0.869318,-2.551136,-1.364268,-1.551136,-0.712753,-2.608586,-1.426768,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6986,-9.000000,0.000000,-9.000000,-9.000000,-2.250000,-0.250000,-1.000000,-0.250000,-1.000000,-1.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6987,-1.350000,-0.555556,-4.450000,-7.688889,-6.950000,-1.505556,-3.600000,-0.088889,-0.605556,-0.738889,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [17]:
kelasdata.sum(axis=1)

0       1
1       1
2       1
3       1
4       1
       ..
6984    1
6985    1
6986    1
6987    1
6988    1
Length: 6989, dtype: int64

# SAE MODEL

In [21]:
#Fungsi model SAE
def sae_model(xt, xv = None, EPOCHS = 100,BATCH_SIZE = 32, opt = "adam",
              hl_node = 1024, lr = 0.01,af = "relu",num_layers = 3, do=0.5, fr_node = 0.5,
              verbose = 0,return_fe = False):
  #Setting result placeholders
  xt_ae = [] ;xv_ae = [] ; w_ae = []
  #If validation set is not present, use train set as validation set
  if xv is None :
    xv = xt.copy()
  opt = tf.keras.optimizers.get(opt) #Set optimizer
  K.set_value(opt.learning_rate, lr) #Set learning rate

  #Stacked Autoencoder architecture
  for n_layers in range(num_layers):
    #Autoencoder
    inp = Input(shape=(xt.shape[1],))
    #Apply Dropout
    hidden_layer = Dropout(do)(inp)
    #Layer encoder (jumlah layer sesuai dengan n_layers)
    enc = Dense(int(hl_node*(fr_node**n_layers)), activation = af)(hidden_layer)  
    #Layer Decoder
    dec = Dense(xt.shape[1],activation="linear")(enc)
    ae = Model(inp, dec)
    #Compile model
    ae.compile(optimizer=opt, loss='mean_squared_error')
    #EarlyStop jika sudah konvergen 
    es = EarlyStopping(monitor='val_loss', patience=15, verbose=verbose)
    #Latih model
    ae.fit(xt, xt, 
           epochs=EPOCHS,batch_size=BATCH_SIZE, 
           shuffle=True, callbacks = [es] , verbose = verbose,
           validation_data = (xv,xv))
    #Ekstrak Feature extraction
    fe = Model(ae.input, enc)
    #Simpan data hasil latih
    xt = fe.predict(xt) ; xt_ae.append(xt)
    xv = fe.predict(xv) ; xv_ae.append(xv)
    #Simpan bobot hasil latih SAE
    w_ae.append([layer_name for layer_name in ae.layers if "dense" in layer_name.name][0].get_weights())
    if verbose:
      print("Layer {} trained".format(n_layers+1))

  return (w_ae,xv) if return_fe else w_ae

# BUILD DNN

In [22]:
#Fungsi DNN
def dnn_model(xt, n_outputs = 4655, sae_weights = None, EPOCHS = 100,BATCH_SIZE = 32, opt = "adam",
              hl_node = 1024, lr = 0.01,af = "relu",num_layers = 3, do=0.5, fr_node = 0.5):
  opt = tf.keras.optimizers.get(opt) #Set optimizer
  K.set_value(opt.learning_rate, lr) #Set learning rate
  
  #Model architecture
  input_layer = Input(shape=(xt.shape[1],))
  hidden_layer = BatchNormalization()(input_layer)
  hidden_layer = Dropout(do)(hidden_layer)
#Set jumlah hidden layer
  for n_layers in range(num_layers):
    hidden_layer = Dense(int(hl_node*(fr_node**n_layers)), activation = af)(hidden_layer)
    hidden_layer = BatchNormalization()(hidden_layer)
    hidden_layer = Dropout(do)(hidden_layer)
  output_layer = Dense(n_outputs, activation = 'sigmoid')(hidden_layer)
#latih model
  dnn = Model(input_layer, output_layer)

  #Latih model DNN dengan bobot SAE (jika bobot ada)
  if sae_weights is not None:
    weights = sae_weights
    dnn_dense = [layer_name for layer_name in dnn.layers if "dense" in layer_name.name]
    for weight_from,weight_to in list(zip(weights,dnn_dense)):
      weight_to.set_weights(weight_from)

  #Compile model
  dnn.compile(optimizer=opt, loss='binary_crossentropy', metrics = [
               tf.keras.metrics.Precision(),
               tf.keras.metrics.Recall()],
               )
  return dnn

In [27]:
scaler_all = MinMaxScaler()
X_pepi = pd.DataFrame(data = scaler_all.fit_transform(X_pepi), columns = X_pepi.columns)
X_pepi

,0,1,2,3,4,5,6,7,8,9,...,590,591,592,593,594,595,596,597,598,599
0,0.604938,0.680297,0.000000,0.309852,0.365079,0.409594,0.446328,0.980121,0.680284,0.624588,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.919297,0.152567,0.693288,0.916163,0.878576,0.215267,0.694614,0.704634,0.404988,0.306307,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.860863,0.564225,0.645287,0.692190,0.803968,0.429197,0.642252,0.000000,0.608976,0.604058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.866969,0.563069,0.580848,0.610964,0.734975,0.446494,0.600138,0.401187,0.881770,0.435445,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.860863,0.564225,0.645287,0.692190,0.803968,0.429197,0.642252,0.000000,0.608976,0.604058,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6984,0.953619,0.931913,0.884686,0.969704,0.871542,0.852994,0.825540,0.840154,0.814680,0.702375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6985,0.913510,0.061902,0.751210,0.936099,0.870418,0.093844,0.684515,0.744958,0.399516,0.397421,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6986,0.000000,0.831461,0.041420,0.338438,0.885714,0.833948,0.796610,0.910543,0.769805,0.577661,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6987,0.850000,0.453552,0.526036,0.434814,0.646984,0.000000,0.267797,0.968193,0.860604,0.687939,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [33]:
#Train SAE-DNN
#Variabel untuk simpan hasil
acc_results = list()
f1_results = list()
prec_results = list()
rec_results = list()
#Latih SAE 
print("finding sae weights....")
ti0 = time.time()
sae_weights = sae_model(xt = X_pepi)
ti1 = time.time()
print('done, processing time:', ti1-ti0)
i=0
t0 = time.time()
#Inisialisasi CV
np.random.seed(123)
cv = IterativeStratification(n_splits=5, random_state = 123)
#Mulai latih DNN dengan bobot hasil SAE untuk tiap CV
for train_ix, test_ix in cv.split(X_pepi,Y):
    #Bagi data menjadi train, test
    X_train1, X_test1 = X_pepi.iloc[train_ix,:], X_pepi.iloc[test_ix,:]
    y_train1, y_test1 = Y[train_ix], Y[test_ix]
    #minmax scaler
    scaler = MinMaxScaler()
    X_train_mm = pd.DataFrame(data = scaler.fit_transform(X_train1), columns = X_train1.columns)
    X_test_mm = pd.DataFrame(data = scaler.transform(X_test1), columns = X_test1.columns)   
    # define model dengan bobot SAE. Jika tidak memakai bobot SAE, sae_weights = None
    model = dnn_model(xt = X_train_mm, sae_weights = sae_weights)
    # latih model
    model.fit(X_train_mm, y_train1, verbose=False, epochs=100)
    # Prediksi test 
    yhat = model.predict(X_test_mm)
    # Bulatkan hasil prediksi (probabilitas)
    yhat = yhat.round()
    # Hitung metrik
    acc = accuracy_score(y_test1, yhat)
    f1 = f1_score(y_test1, yhat, average='samples')
    prec = precision_score(y_test1, yhat, average='samples')
    rec = recall_score(y_test1, yhat, average='samples')
    # Simpan hasil
    print("CV number: ", i)
    print('accuracy of :>%.3f' % acc)
    print('F1 of :>%.3f' % f1)
    print('Precision of :>%.3f' % prec)
    print('Recall of :>%.3f' % rec)
    print('y', yhat.sum())
    acc_results.append(acc)
    f1_results.append(f1)
    prec_results.append(prec)
    rec_results.append(rec)
    i=i+1
t1 = time.time()
print("waktu proses", t1-t0)
print("Accuracy array:", acc_results)
print("F1 array:", f1_results)
print("Precision array:", prec_results)
print("Recall array:", rec_results)
#save model
model.save("SAE-DNN PubChem pepi.h5")
        

finding sae weights....
done, processing time: 36.105360984802246


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  0
accuracy of :>0.005
F1 of :>0.005
Precision of :>0.005
Recall of :>0.005
y 39.0


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  1
accuracy of :>0.000
F1 of :>0.000
Precision of :>0.000
Recall of :>0.000
y 0.0


...

CV number:  2
accuracy of :>0.001
F1 of :>0.001
Precision of :>0.001
Recall of :>0.001
y 2.0


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  3
accuracy of :>0.003
F1 of :>0.003
Precision of :>0.003
Recall of :>0.003
y 6.0


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  4
accuracy of :>0.001
F1 of :>0.001
Precision of :>0.001
Recall of :>0.001
y 1.0
waktu proses 522.5508360862732
Accuracy array: [0.005043227665706052, 0.0, 0.0007057163020465773, 0.002881844380403458, 0.0007320644216691069]
F1 array: [0.005043227665706052, 0.0, 0.0007057163020465773, 0.002881844380403458, 0.0007320644216691069]
Precision array: [0.005043227665706052, 0.0, 0.0007057163020465773, 0.002881844380403458, 0.0007320644216691069]
Recall array: [0.005043227665706052, 0.0, 0.0007057163020465773, 0.002881844380403458, 0.0007320644216691069]


In [34]:
print('===================================')
print('SAE-DNN PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(acc_results), np.std(acc_results)))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(f1_results), np.std(f1_results)))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(prec_results), np.std(prec_results)))
print('Recall      : {0:.5f}±{1:.3f}'.format(np.mean(rec_results), np.std(rec_results)))
print('===================================')


SAE-DNN PERFORMANCE
Accuracy    : 0.00187±0.002
F1 Score    : 0.00187±0.002
Precision   : 0.00187±0.002
Recall      : 0.00187±0.002


# TUNING PARAM

In [35]:
#Fungsi model untuk tuning
def build_model(hp):
    #Isi parameter yang akan dituning
    params = {
              'hl_node' : hp.Choice('units',values = [100,200,300,400,500,600]),
#               'af' : hp.Choice('activation',values = ["relu"]),
              'lr' : hp.Choice('learning_rate',values=[x for x in np.linspace(0.01,0.1,10)]),
#               'opt' : hp.Choice('optimizer',values=["adam"]),
              'num_layers' : hp.Choice('num_layers',values=[2,3,4,5]),
              'do' : hp.Choice('dropout_rate',values=[x for x in np.linspace(0.1,0.5,5)]),
              'fr_node' : hp.Choice('fraction_node',values=[0.5,0.66,0.75])
              }
    #Latih model SAE
    sae_weights = sae_model(xt = X_pepi, EPOCHS= 100,**params)
    #Latih model DNN dengan bobot SAE
    sae_dnn = dnn_model(X_train_mm, sae_weights=sae_weights, EPOCHS= 100,**params)
    return sae_dnn

In [36]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience = 30)
# Fungsi Bayesian Optimization di Keras Tuner.
tuner = BayesianOptimization(build_model,
    # Metrik yang dicari optimalnya
    objective= kt.Objective("val_loss", direction="min"), 
    # Jumlah percobaan
    max_trials=35,
    executions_per_trial=1,
    # Folder simpan hasil tuning
    directory='E:\Projek Pak Wisnu\Tuning Folder',
    project_name='SAE-DNN_TUNING_PEPI')
#Jalankan Keras Tuner u
tuner.search(X_train_mm, y_train1, epochs=100, validation_data=(X_test_mm, y_test1),callbacks=[stop_early])
#Tampilkan hasil terbaik
tuner.results_summary()

Trial 35 Complete [00h 01m 00s]
val_loss: 0.002298167906701565

Best val_loss So Far: 0.0022901722695678473
Total elapsed time: 00h 48m 17s
INFO:tensorflow:Oracle triggered exit


INFO:tensorflow:Oracle triggered exit


Results summary
Results in E:\Projek Pak Wisnu\Tuning Folder\SAE-DNN_TUNING_PEPI
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.0022901722695678473
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.0022960975766181946
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.002297885250300169
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.002298167906701565
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.002298504114151001
Trial summary
Hyperparameters:
units: 100
learning_rate: 0.01
num_layers: 2
dropout_rate: 0.2
fraction_node: 0.5
Score: 0.0022999753709882498
Tri

In [38]:
#Train SAE-DNN with BEST HP 
acc_results_tuned = list()
f1_results_tuned = list()
prec_results_tuned = list()
rec_results_tuned = list()
# define evaluation procedure

print("finding sae weights....")
ti0 = time.time()

sae_weigths_tuned = sae_model(xt = X_pepi, hl_node =100, af = "relu", lr = 0.01, opt =  "adam", num_layers = 2,
                              do = 0.2, fr_node= 0.5)

ti1 = time.time()
print('done, processing time:', ti1-ti0)
i=0
t0 = time.time()
# enumerate folds
np.random.seed(123)
cv = IterativeStratification(n_splits=10, random_state = 123)
for train_ix, test_ix in cv.split(X_pepi,Y):
    # prepare data
    X_train1, X_test1 = X_pepi.iloc[train_ix,:], X_pepi.iloc[test_ix,:]
    y_train1, y_test1 = Y[train_ix], Y[test_ix]
    #minmax scaler
    scaler = MinMaxScaler()
    X_train_mm = pd.DataFrame(data = scaler.fit_transform(X_train1), columns = X_train1.columns)
    X_test_mm = pd.DataFrame(data = scaler.transform(X_test1), columns = X_test1.columns)  
    # define model
    model_tuned = dnn_model(xt = X_train_mm, sae_weights = sae_weigths_tuned,  hl_node =100, af = "relu", lr = 0.01, opt =  "adam", num_layers = 2,
                              do = 0.2, fr_node= 0.5)
    # fit model
    model_tuned.fit(X_train_mm, y_train1, verbose=False, epochs=100)
    # make a prediction on the test set
    yhat = model_tuned.predict(X_test_mm)
    # round probabilities to class labels
    yhat = yhat.round()
    # calculate metrics
    acc = accuracy_score(y_test1, yhat)
    f1 = f1_score(y_test1, yhat, average='samples')
    prec = precision_score(y_test1, yhat, average='samples')
    rec = recall_score(y_test1, yhat, average='samples')
    # store result
    print("CV number: ", i)
    print('accuracy of :>%.3f' % acc)
    print('F1 of :>%.3f' % f1)
    print('Precision of :>%.3f' % prec)
    print('Recall of :>%.3f' % rec)
    acc_results_tuned.append(acc)
    f1_results_tuned.append(f1)
    prec_results_tuned.append(prec)
    rec_results_tuned.append(rec)
    i=i+1
t1 = time.time()
print("waktu proses: ", t1-t0)
print("Accuracy array:", acc_results_tuned)
print("F1 array:", f1_results_tuned)
print("Precision array:", prec_results_tuned)
print("Recall array:", rec_results_tuned)
#save model
model_tuned.save("SAE-DNN_PEPI_TUNED.h5")

finding sae weights....
done, processing time: 25.655171394348145


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  0
accuracy of :>0.000
F1 of :>0.000
Precision of :>0.000
Recall of :>0.000


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  1
accuracy of :>0.000
F1 of :>0.000
Precision of :>0.000
Recall of :>0.000


...

CV number:  7
accuracy of :>0.000
F1 of :>0.000
Precision of :>0.000
Recall of :>0.000


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  8
accuracy of :>0.000
F1 of :>0.000
Precision of :>0.000
Recall of :>0.000


D:\Anaconda3\envs\py37\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


CV number:  9
accuracy of :>0.001
F1 of :>0.001
Precision of :>0.001
Recall of :>0.001
waktu proses:  1008.7592604160309
Accuracy array: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013812154696132596, 0.0, 0.0, 0.0014184397163120568]
F1 array: [0.0, 0.0, 0.0009537434430138292, 0.0, 0.0, 0.0, 0.0013812154696132596, 0.0, 0.0, 0.0014184397163120568]
Precision array: [0.0, 0.0, 0.000715307582260372, 0.0, 0.0, 0.0, 0.0013812154696132596, 0.0, 0.0, 0.0014184397163120568]
Recall array: [0.0, 0.0, 0.001430615164520744, 0.0, 0.0, 0.0, 0.0013812154696132596, 0.0, 0.0, 0.0014184397163120568]


In [39]:
print('===================================')
print('===================================')
print('SAE-DNN TUNED PERFORMANCE')
print('Accuracy    : {0:.5f}±{1:.3f}'.format(np.mean(acc_results_tuned), np.std(acc_results_tuned)))
print('F1 Score    : {0:.5f}±{1:.3f}'.format(np.mean(f1_results_tuned), np.std(f1_results_tuned)))
print('Precision   : {0:.5f}±{1:.3f}'.format(np.mean(prec_results_tuned), np.std(prec_results_tuned)))
print('Recall      : {0:.5f}±{1:.3f}'.format(np.mean(rec_results_tuned), np.std(rec_results_tuned)))
print('===================================')
print('===================================')


SAE-DNN TUNED PERFORMANCE
Accuracy    : 0.00028±0.001
F1 Score    : 0.00038±0.001
Precision   : 0.00035±0.001
Recall      : 0.00042±0.001
